In [ ]:
!pip install giotto-tda

You should consider upgrading via the '/Users/rasuharu/Dev/kaggle/jpx_tokyo_market_prediction/.venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd
from gtda.time_series import PearsonDissimilarity
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Amplitude

In [170]:
df = pd.read_csv("../Input/train_files/stock_prices.csv")

In [171]:
df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


In [172]:
df['Datetime'] = pd.to_datetime(df['Date'])

In [173]:
pivot = df.pivot(index='Date', columns='SecuritiesCode', values='Close')
pivot.fillna(method='ffill', inplace=True)
pivot.interpolate(limit_direction='both', inplace=True)
fillna_df = pivot.stack().reset_index().sort_values(by=['Date', 'SecuritiesCode'])

In [174]:
df.shape

(2332531, 13)

In [175]:
df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Datetime
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,2017-01-04
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,2017-01-04
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,2017-01-04
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,2017-01-04
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,2017-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816,2021-12-03
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478,2021-12-03
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302,2021-12-03
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098,2021-12-03


In [176]:
fillna_df.columns = ['Date', 'SecuritiesCode', 'Close_fillna']

In [177]:
fillna_df

,Date,SecuritiesCode,Close_fillna
0,2017-01-04,1301,2742.0
1,2017-01-04,1332,571.0
2,2017-01-04,1333,3210.0
3,2017-01-04,1375,2090.0
4,2017-01-04,1376,1550.0
...,...,...,...
2403995,2021-12-03,9990,528.0
2403996,2021-12-03,9991,794.0
2403997,2021-12-03,9993,1645.0
2403998,2021-12-03,9994,2389.0


In [178]:
f_df = df.merge(fillna_df, on=['Date', 'SecuritiesCode'], how='left')

In [179]:
f_df[(f_df['Close'] != f_df['Close_fillna']) & (f_df['SecuritiesCode'] == 3540)]

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Datetime,Close_fillna
401,20170104_3540,2017-01-04,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN,2017-01-04,2050.0
2266,20170105_3540,2017-01-05,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN,2017-01-05,2050.0
4131,20170106_3540,2017-01-06,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN,2017-01-06,2050.0
5996,20170110_3540,2017-01-10,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN,2017-01-10,2050.0
7861,20170111_3540,2017-01-11,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN,2017-01-11,2050.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435744,20171211_3540,2017-12-11,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,True,NaN,2017-12-11,2050.0
437631,20171212_3540,2017-12-12,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,True,NaN,2017-12-12,2050.0
439519,20171213_3540,2017-12-13,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,True,NaN,2017-12-13,2050.0
441408,20171214_3540,2017-12-14,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,True,1.119512,2017-12-14,2050.0


In [180]:
f_df[['SecuritiesCode', 'Close_fillna', 'Volume']].isnull().sum()

SecuritiesCode    0
Close_fillna      0
Volume            0
dtype: int64

In [250]:
df_1301 = f_df[f_df['SecuritiesCode'] == 1301]

In [ ]:
df_1301.shape

In [259]:
X = df_1301[['Close_fillna', 'Volume']].values
y = df_1301['Target'].fillna(0).values

In [260]:
from gtda.time_series import SlidingWindow

window_size = 3
stride = 2

SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X, y)
X_sw, yr

(array([[[ 2738., 17900.],
         [ 2740., 19900.],
         [ 2748., 24200.]],
 
        [[ 2748., 24200.],
         [ 2745.,  9300.],
         [ 2731., 28700.]],
 
        [[ 2731., 28700.],
         [ 2722., 19400.],
         [ 2704., 20100.]],
 
        ...,
 
        [[ 3005.,  6400.],
         [ 2995., 11600.],
         [ 2951., 14300.]],
 
        [[ 2951., 14300.],
         [ 2900., 20500.],
         [ 2911., 10600.]],
 
        [[ 2911., 10600.],
         [ 2933., 15200.],
         [ 2982., 11400.]]]),
 array([-0.00510018, -0.00661278,  0.00297841,  0.01407929,  0.00257353,
         0.00073233,  0.00256692,  0.0047759 , -0.00329791,  0.00220183,
         0.        ,  0.01347905,  0.00821135,  0.00035398,  0.01087338,
         0.00034165,  0.00033956,  0.01480983,  0.00662252,  0.00160772,
         0.00637959,  0.00956938, -0.01875   ,  0.01107595, -0.01098901,
         0.00321027, -0.00160256, -0.02259136,  0.02520436, -0.00166113,
         0.00067981, -0.00512645, -0.004145

In [276]:
X_sw.shape

(600, 3, 2)

In [261]:

PD = PearsonDissimilarity()
X_pd = PD.fit_transform(X_sw)
VR = VietorisRipsPersistence(metric="precomputed")
X_vr = VR.fit_transform(np.nan_to_num(X_pd))  # "precomputed" required on dissimilarity data
Ampl = Amplitude()
X_a = Ampl.fit_transform(X_vr)
X_a

array([[0.        , 0.        ],
       [0.19671913, 0.        ],
       [0.01595124, 0.        ],
       ...,
       [0.25956523, 0.        ],
       [0.15901652, 0.        ],
       [0.11025089, 0.        ]])

In [265]:
from sklearn import set_config
set_config(display='diagram')  # For HTML representations of pipelines

from gtda.pipeline import make_pipeline

pipe = make_pipeline(SW, PD, VR, Ampl)
pipe


Pipeline(steps=[('slidingwindow', SlidingWindow(size=3, stride=2)),
                ('pearsondissimilarity', PearsonDissimilarity()),
                ('vietorisripspersistence',
                 VietorisRipsPersistence(metric='precomputed')),
                ('amplitude', Amplitude())])

In [266]:
from sklearn.ensemble import RandomForestRegressor

RFR = RandomForestRegressor()

pipe = make_pipeline(SW, PD, VR, Ampl, RFR)
pipe

Pipeline(steps=[('slidingwindow', SlidingWindow(size=3, stride=2)),
                ('pearsondissimilarity', PearsonDissimilarity()),
                ('vietorisripspersistence',
                 VietorisRipsPersistence(metric='precomputed')),
                ('amplitude', Amplitude()),
                ('randomforestregressor', RandomForestRegressor())])

In [267]:
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

(array([ 1.68681544e-03, -4.82075630e-03,  3.86877408e-03,  1.02356688e-02,
         1.68681544e-03, -6.55640556e-04,  2.04427591e-03,  1.78470861e-03,
         1.68681544e-03,  3.25168441e-04,  6.02259551e-04,  4.81724356e-03,
        -4.88658487e-03,  3.19318940e-03,  2.93940952e-03,  1.91635697e-03,
         3.65648994e-03,  8.82202718e-03,  1.34498190e-03,  4.87640620e-03,
         4.58827225e-03,  8.07056245e-03, -1.39708539e-02,  1.03377150e-02,
        -7.85131688e-03,  3.35753262e-03,  2.35050635e-03, -1.17476388e-02,
         1.53208514e-02, -5.57158693e-05,  1.68681544e-03, -5.34175495e-03,
         1.68681544e-03,  6.25749720e-03,  4.24888611e-03,  5.28624717e-03,
         8.72252528e-04, -1.16102315e-03, -5.96374781e-03,  1.02485546e-02,
         6.28733585e-03, -7.29350866e-03,  1.68681544e-03, -3.92677263e-03,
         1.18722176e-02,  3.60913754e-03, -1.26147348e-02, -5.91892310e-03,
        -2.16409972e-03,  1.55055703e-03, -2.51615716e-03,  9.27049369e-04,
         1.2

In [273]:
X.shape

(1202, 2)

In [274]:
y_pred.shape

(600,)

In [297]:
X = df_1301['Close_fillna'].values
y = df_1301['Target'].fillna(0).values

In [316]:
from gtda.time_series import TakensEmbedding

window_size = 5
stride = 1

SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X, y)
X_sw, yr

(array([[2742., 2738., 2740., 2748., 2745.],
        [2738., 2740., 2748., 2745., 2731.],
        [2740., 2748., 2745., 2731., 2722.],
        ...,
        [3005., 2995., 2951., 2900., 2911.],
        [2995., 2951., 2900., 2911., 2933.],
        [2951., 2900., 2911., 2933., 2982.]]),
 array([-0.0032955 , -0.00661278, -0.0066568 , ...,  0.01670644,
        -0.0036888 ,  0.03163918]))

In [293]:
time_delay = 1
dimension = 2

TE = TakensEmbedding(time_delay=time_delay, dimension=dimension)
X_te = TE.fit_transform(X_sw)
X_te

array([[[ 2738., 17900.,  2740., ...,  9300.,  2731., 28700.]],

       [[ 2748., 24200.,  2745., ..., 19400.,  2704., 20100.]],

       [[ 2731., 28700.,  2722., ..., 18400.,  2694., 12100.]],

       ...,

       [[ 3045.,  5500.,  3005., ..., 11600.,  2951., 14300.]],

       [[ 3005.,  6400.,  2995., ..., 20500.,  2911., 10600.]],

       [[ 2951., 14300.,  2900., ..., 15200.,  2982., 11400.]]])

In [294]:
VR = VietorisRipsPersistence()  # No "precomputed" for point clouds
Ampl = Amplitude()
RFR = RandomForestRegressor()

pipe = make_pipeline(SW, TE, VR, Ampl, RFR)
pipe

Pipeline(steps=[('slidingwindow', SlidingWindow(size=5, stride=2)),
                ('takensembedding', TakensEmbedding()),
                ('vietorisripspersistence', VietorisRipsPersistence()),
                ('amplitude', Amplitude()),
                ('randomforestregressor', RandomForestRegressor())])

In [298]:
X

array([2742., 2738., 2740., ..., 2911., 2933., 2982.])

In [299]:
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

(array([-4.54544181e-03,  4.33605291e-03,  3.40297862e-03, -3.23767782e-03,
        -5.47443611e-03,  2.72963724e-03,  5.00856034e-03, -2.05188180e-03,
         2.72963724e-03, -1.28355539e-03,  9.07714307e-03,  6.90912776e-03,
        -2.11208475e-03,  6.64854102e-03,  6.72567075e-03, -6.81343064e-03,
         8.94254100e-03, -2.04889723e-03, -2.04889723e-03,  1.57442154e-03,
         4.09658838e-03,  4.28266848e-04,  3.47077347e-03, -4.28363912e-03,
         2.72271443e-03, -1.70779139e-02, -1.13760896e-02,  1.01914622e-02,
         1.01914622e-02, -2.92379605e-03, -4.65253209e-03, -4.42112976e-03,
         6.23146337e-03,  7.72718852e-03,  4.92328201e-03,  3.74298955e-04,
         3.54467177e-03,  2.88115317e-04,  5.76662780e-03, -4.54656216e-03,
        -4.54656216e-03,  6.17999871e-03,  1.01778222e-03,  8.04179379e-03,
        -2.73649057e-03, -2.73649057e-03, -5.22379956e-03,  2.94906376e-03,
        -2.62853839e-03, -5.34470488e-04,  1.02444952e-03, -2.50449374e-03,
        -4.8

In [300]:
y_pred.shape

(599,)

In [283]:
from gtda.time_series import SingleTakensEmbedding

STE = SingleTakensEmbedding(parameters_type="search", time_delay=2, dimension=3)
X_ste, yr = STE.fit_transform_resample(X, y)
X_ste, yr

(array([[2742., 2740., 2745.],
        [2738., 2748., 2731.],
        [2740., 2745., 2722.],
        ...,
        [3005., 2951., 2911.],
        [2995., 2900., 2933.],
        [2951., 2911., 2982.]]),
 array([-0.0032955 , -0.00661278, -0.0066568 , ...,  0.01670644,
        -0.0036888 ,  0.03163918]))

In [314]:
window_size = 5
stride = 1

SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X_ste, yr)
X_sw, yr

(array([[[2742., 2740., 2745.],
         [2738., 2748., 2731.],
         [2740., 2745., 2722.],
         [2748., 2731., 2704.],
         [2745., 2722., 2686.]],
 
        [[2738., 2748., 2731.],
         [2740., 2745., 2722.],
         [2748., 2731., 2704.],
         [2745., 2722., 2686.],
         [2731., 2704., 2694.]],
 
        [[2740., 2745., 2722.],
         [2748., 2731., 2704.],
         [2745., 2722., 2686.],
         [2731., 2704., 2694.],
         [2722., 2686., 2699.]],
 
        ...,
 
        [[3045., 3045., 3005.],
         [3040., 3005., 2995.],
         [3045., 3005., 2951.],
         [3005., 2995., 2900.],
         [3005., 2951., 2911.]],
 
        [[3040., 3005., 2995.],
         [3045., 3005., 2951.],
         [3005., 2995., 2900.],
         [3005., 2951., 2911.],
         [2995., 2900., 2933.]],
 
        [[3045., 3005., 2951.],
         [3005., 2995., 2900.],
         [3005., 2951., 2911.],
         [2995., 2900., 2933.],
         [2951., 2911., 2982.]]]),
 array(

In [315]:
X_sw.shape

(1194, 5, 3)

In [313]:
X_sw.shape

(597, 5, 3)

In [285]:
VR = VietorisRipsPersistence()  # No "precomputed" for point clouds
Ampl = Amplitude()
RFR = RandomForestRegressor()

pipe = make_pipeline(STE, SW, VR, Ampl, RFR)
pipe

Pipeline(steps=[('singletakensembedding',
                 SingleTakensEmbedding(dimension=3, time_delay=2)),
                ('slidingwindow', SlidingWindow(size=5, stride=2)),
                ('vietorisripspersistence', VietorisRipsPersistence()),
                ('amplitude', Amplitude()),
                ('randomforestregressor', RandomForestRegressor())])

In [286]:
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

(array([ 9.68483883e-03, -1.02828168e-03, -1.02828168e-03,  3.02833108e-03,
         5.91230978e-03, -5.14859418e-04,  1.79503398e-03,  5.98615430e-03,
         1.03483065e-02,  3.16523891e-03,  2.24048402e-03,  5.32689596e-03,
         5.32689596e-03, -1.25951359e-04,  7.82893518e-03,  7.82893518e-03,
         7.82893518e-03,  7.48068982e-03,  7.48068982e-03, -2.68234555e-03,
        -2.68234555e-03, -8.69400435e-03,  7.62514284e-03,  1.14673044e-03,
        -6.82342686e-03,  1.24672557e-02,  1.24672557e-02,  1.45391749e-04,
        -2.23517573e-03,  3.47429929e-04,  7.84511898e-03,  7.84511898e-03,
        -2.03946424e-03,  2.28121172e-03,  2.28121172e-03,  4.22012982e-03,
         4.22012982e-03, -3.18971545e-04, -8.16256832e-03,  4.23403378e-03,
         4.23403378e-03,  1.10192924e-02,  1.10192924e-02, -8.19765169e-03,
        -5.82468431e-03, -5.82468431e-03,  6.36181724e-03,  3.70702997e-03,
        -6.35506703e-03, -2.11165975e-03, -9.83501433e-03,  1.33781394e-03,
         9.5

In [287]:
y_pred.shape

(597,)

In [372]:
from functools import partial
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.base import clone

mean = FunctionTransformer(partial(np.mean, axis=1, keepdims=True))
var = FunctionTransformer(partial(np.var, axis=1, keepdims=True))

pipe_topology = make_pipeline(TE, VR, Ampl)

feature_union = FeatureUnion([("window_mean", mean),
                              ("window_variance", var),
                              ("window_topology", pipe_topology)])
    
# pipe = make_pipeline(SW, feature_union, RFR)
pipe = make_pipeline(SW, feature_union)
pipe

Pipeline(steps=[('slidingwindow', SlidingWindow(size=5)),
                ('featureunion',
                 FeatureUnion(transformer_list=[('window_mean',
                                                 FunctionTransformer(func=functools.partial(<function mean at 0x10e1b1700>, axis=1, keepdims=True))),
                                                ('window_variance',
                                                 FunctionTransformer(func=functools.partial(<function var at 0x10e1b1a60>, axis=1, keepdims=True))),
                                                ('window_topology',
                                                 Pipeline(steps=[('takensembedding',
                                                                  TakensEmbedding()),
                                                                 ('vietorisripspersistence',
                                                                  VietorisRipsPersistence()),
                                                                 ('amplitude',
                                                                  Amplitude())]))]))])

In [373]:
X_trans = pipe.transform(X)

In [377]:
fig = px.line(title='Pipeline transform X, and original y')
fig.add_scatter(y=X_trans[:, 2], name='X_trans[:, 2]')
fig.show()

In [376]:
fig = px.line(title='Pipeline transform X, and original y')
fig.add_scatter(y=X_trans[:, 3], name='X_trans[:, 3]')
fig.show()

In [381]:
X_trans[484:487]

array([[ 2832.4       , 19480.64      ,   741.38636172,     0.        ],
       [ 2777.2       , 15925.76      ,   741.38636172,     0.        ],
       [ 2759.4       , 10578.64      ,   802.34006797,     0.        ]])

In [382]:
y[4:][484:487]

array([ 0.06483883,  0.00278358, -0.01040944])

In [375]:
fig = px.line(title='Original y')
fig.add_scatter(y=y[4:], name='y')
fig.show()

In [354]:
mean = FunctionTransformer(partial(np.mean, axis=1, keepdims=True))
var = FunctionTransformer(partial(np.var, axis=1, keepdims=True))

time_delay = 1
dimension = 2

TE = TakensEmbedding(time_delay=time_delay, dimension=dimension)

window_size = 5
stride = 1

SW = SlidingWindow(size=window_size, stride=stride)

pipe_topology = make_pipeline(TE, VR, Ampl)
# pipe_topology = make_pipeline(TE, VR)

feature_union = FeatureUnion([("window_mean", mean),
                              ("window_variance", var),
                              ("window_topology", pipe_topology)])
    
# pipe = make_pipeline(SW, feature_union, RFR)
pipe = make_pipeline(SW, feature_union)
pipe

Pipeline(steps=[('slidingwindow', SlidingWindow(size=5)),
                ('featureunion',
                 FeatureUnion(transformer_list=[('window_mean',
                                                 FunctionTransformer(func=functools.partial(<function mean at 0x10e1b1700>, axis=1, keepdims=True))),
                                                ('window_variance',
                                                 FunctionTransformer(func=functools.partial(<function var at 0x10e1b1a60>, axis=1, keepdims=True))),
                                                ('window_topology',
                                                 Pipeline(steps=[('takensembedding',
                                                                  TakensEmbedding()),
                                                                 ('vietorisripspersistence',
                                                                  VietorisRipsPersistence()),
                                                                 ('amplitude',
                                                                  Amplitude())]))]))])

In [357]:
target_index = df_1301.shape[0] // 2
X_small = df_1301['Close_fillna'].values[target_index:]
y_small = df_1301['Target'].fillna(0).values[target_index:]

In [358]:
X = df_1301['Close_fillna'].values
y = df_1301['Target'].fillna(0).values

In [360]:
X_small.shape

(601,)

In [359]:
X.shape

(1202,)

In [361]:
pipe.fit(X_small, y_small)

Pipeline(steps=[('slidingwindow', SlidingWindow(size=5)),
                ('featureunion',
                 FeatureUnion(transformer_list=[('window_mean',
                                                 FunctionTransformer(func=functools.partial(<function mean at 0x10e1b1700>, axis=1, keepdims=True))),
                                                ('window_variance',
                                                 FunctionTransformer(func=functools.partial(<function var at 0x10e1b1a60>, axis=1, keepdims=True))),
                                                ('window_topology',
                                                 Pipeline(steps=[('takensembedding',
                                                                  TakensEmbedding()),
                                                                 ('vietorisripspersistence',
                                                                  VietorisRipsPersistence()),
                                                                 ('amplitude',
                                                                  Amplitude())]))]))])

In [362]:
pipe.transform(X)

array([[2742.6       ,   12.64      ,    7.1586008 ,    0.        ],
       [2740.4       ,   34.64      ,   11.3949405 ,    0.        ],
       [2737.2       ,   90.96      ,   19.49279796,    0.        ],
       ...,
       [2952.4       , 1808.64      ,  159.58356451,    0.        ],
       [2938.        , 1123.2       ,  159.58356451,    0.        ],
       [2935.4       ,  853.84      ,  113.64919067,    0.        ]])

In [363]:
X_trans

array([[2.74260000e+03, 1.26400000e+01, 7.48788202e+00, 7.83302877e-01],
       [2.74040000e+03, 3.46400000e+01, 1.10598956e+01, 0.00000000e+00],
       [2.73720000e+03, 9.09600000e+01, 1.98954301e+01, 0.00000000e+00],
       ...,
       [2.95240000e+03, 1.80864000e+03, 1.59585073e+02, 0.00000000e+00],
       [2.93800000e+03, 1.12320000e+03, 1.59585073e+02, 0.00000000e+00],
       [2.93540000e+03, 8.53840000e+02, 1.13735266e+02, 0.00000000e+00]])

In [367]:
X_trans.shape

(1198, 4)

In [370]:
y[4:].shape

(1198,)

In [365]:
fig = px.line(title='Pipeline transform X, and original y')
fig.add_scatter(y=X_trans[:, 2], name='X_trans[:, 2]')
fig.add_scatter(y=X_trans[:, 3], name='X_trans[:, 3]')
fig.show()

In [371]:
fig = px.line(title='Original y')
fig.add_scatter(y=y[4:], name='y')
fig.show()

In [319]:
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

(array([-0.00064056, -0.0027228 , -0.00525716, ...,  0.01172612,
        -0.00252753,  0.02067238]),
 0.8388754708629897)

In [332]:
X

array([2742., 2738., 2740., ..., 2911., 2933., 2982.])

In [324]:
# pipe.fit(X, y)
# y_pred = pipe.predict(X)
# score = pipe.score(X, y)
# y_pred, score

X_trans = pipe.fit_transform(X)

In [327]:
X_trans

array([[2.74260000e+03, 1.26400000e+01, 7.48788202e+00, 7.83302877e-01],
       [2.74040000e+03, 3.46400000e+01, 1.10598956e+01, 0.00000000e+00],
       [2.73720000e+03, 9.09600000e+01, 1.98954301e+01, 0.00000000e+00],
       ...,
       [2.95240000e+03, 1.80864000e+03, 1.59585073e+02, 0.00000000e+00],
       [2.93800000e+03, 1.12320000e+03, 1.59585073e+02, 0.00000000e+00],
       [2.93540000e+03, 8.53840000e+02, 1.13735266e+02, 0.00000000e+00]])

In [321]:
y[:20]

array([ 0.00073046,  0.00291971, -0.0010917 , -0.00510018, -0.0032955 ,
       -0.00661278, -0.0066568 ,  0.00297841,  0.00185598,  0.01407929,
       -0.00621118,  0.00257353,  0.00146681,  0.00073233, -0.00219539,
        0.00256692, -0.00438917,  0.0047759 , -0.00219378, -0.00329791])

In [322]:
y_pred[:20]

array([-6.40563067e-04, -2.72280273e-03, -5.25716445e-03,  2.42435440e-03,
        3.21780777e-03,  1.06318345e-02, -5.07788920e-03,  1.78553104e-03,
        1.27715008e-03,  1.22759521e-03, -1.80063357e-03,  2.83682854e-03,
       -1.63742067e-03,  3.38807486e-03, -4.48097906e-04,  9.33602845e-05,
        8.72076181e-04,  2.70486692e-03,  1.71845568e-03,  2.85183804e-04])

In [307]:
X.shape

(1202,)

In [320]:
y_pred.shape

(1198,)

In [343]:
from gtda.homology import WeakAlphaPersistence

mean = FunctionTransformer(partial(np.mean, axis=1, keepdims=True))
var = FunctionTransformer(partial(np.var, axis=1, keepdims=True))

window_size = 5
stride = 1

SW = SlidingWindow(size=window_size, stride=stride)

time_delay = 1
dimension = 2

TE = TakensEmbedding(time_delay=time_delay, dimension=dimension)

homology_dimensions = (0, 1, 2)
WA = WeakAlphaPersistence(homology_dimensions=homology_dimensions)

# pipe_topology = make_pipeline(TE, VR, Ampl)
pipe_topology = make_pipeline(TE, WA, Ampl)

feature_union = FeatureUnion([("window_mean", mean),
                              ("window_variance", var),
                              ("window_topology", pipe_topology)])
    
# pipe = make_pipeline(SW, feature_union, RFR)
pipe = make_pipeline(SW, feature_union)
pipe

Pipeline(steps=[('slidingwindow', SlidingWindow(size=5)),
                ('featureunion',
                 FeatureUnion(transformer_list=[('window_mean',
                                                 FunctionTransformer(func=functools.partial(<function mean at 0x10e1b1700>, axis=1, keepdims=True))),
                                                ('window_variance',
                                                 FunctionTransformer(func=functools.partial(<function var at 0x10e1b1a60>, axis=1, keepdims=True))),
                                                ('window_topology',
                                                 Pipeline(steps=[('takensembedding',
                                                                  TakensEmbedding()),
                                                                 ('weakalphapersistence',
                                                                  WeakAlphaPersistence(homology_dimensions=(0,
                                                                                                            1,
                                                                                                            2))),
                                                                 ('amplitude',
                                                                  Amplitude())]))]))])

In [344]:
X_trans2 = pipe.fit_transform(X)

IndexError: index 6 is out of bounds for axis 0 with size 6

In [302]:
X = df_1301[['Close_fillna', 'Volume']].values
y = df_1301['Target'].fillna(0).values

In [291]:
time_delay = 1
dimension = 2

TE = TakensEmbedding(time_delay=time_delay, dimension=dimension)
X_te = TE.fit_transform(X_sw)

In [ ]:
VR = VietorisRipsPersistence(metric="precomputed")
X_vr = VR.fit_transform(X_pd)  # "precomputed" required on dissimilarity data
Ampl = Amplitude()
X_a = Ampl.fit_transform(X_vr)

In [290]:
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

ValueError: Not enough time stamps (2) to produce at least one 9-dimensional vector under the current choice of time delay (3).

In [ ]:
# VR = VietorisRipsPersistence(metric="precomputed")
# X_vr = VR.fit_transform(X_pd, force_all_finite=True)  # "precomputed" required on dissimilarity data
# Ampl = Amplitude()
# X_a = Ampl.fit_transform(X_vr)
# X_a

In [ ]:
!pip install openml

     |████████████████████████████████| 119 kB 6.1 MB/s            
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
     |████████████████████████████████| 75 kB 17.6 MB/s            
     |████████████████████████████████| 20.2 MB 10.1 MB/s            
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Created wheel for openml: filename=openml-0.12.2-py3-none-any.whl size=137327 sha256=93235adcaf8f233aad2c6aded0fba47fd68536b350ea2874cbafedeca5e230f8
  Stored in directory: /Users/rasuharu/Library/Caches/pip/wheels/bc/16/10/f542aa38a32dfbdec03e4bb3c68b69dbbae98a013107333b39
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11731 sha256=5e9578478f4aba7f871c9cdd9dcede2fc4c637f48ec5f1

In [ ]:
# Import the gtda modules
from gtda.time_series import Resampler, SlidingWindow, takens_embedding_optimal_parameters, \
    TakensEmbedding, PermutationEntropy
from gtda.homology import WeakAlphaPersistence, VietorisRipsPersistence
from gtda.diagrams import Scaler, Filtering, PersistenceEntropy, BettiCurve, PairwiseDistance
from gtda.graphs import KNeighborsGraph, GraphGeodesicDistance

from gtda.pipeline import Pipeline

import numpy as np
from sklearn.metrics import pairwise_distances

import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# gtda plotting functions
from gtda.plotting import plot_heatmap

# Import data from openml
import openml

# Plotting functions
from gtda.plotting import plot_point_cloud

In [ ]:
# Plotting the trajectories of the Lorenz system
from openml.datasets.functions import get_dataset

point_cloud = get_dataset(42182).get_data(dataset_format='array')[0]
plot_point_cloud(point_cloud)

In [ ]:
point_cloud.shape

(4942, 4)

In [ ]:
point_cloud

array([[  1.       , -10.       ,  10.       ,   5.       ],
       [ -0.1      ,  -9.95     ,   9.633333 ,   5.       ],
       [ -1.085    ,  -9.845866 ,   9.3863945,   5.       ],
       ...,
       [ -6.4207535,  -8.754583 ,   9.600212 ,  15.       ],
       [ -6.654136 ,  -9.013744 ,   9.906317 ,  15.       ],
       [ -6.890097 ,  -9.2625475,  10.241936 ,  15.       ]],
      dtype=float32)

In [ ]:
# Selecting the z-axis and the label rho
X = point_cloud[:, 2]
y = point_cloud[:, 3]

In [ ]:
fig = px.line(title='Trajectory of the Lorenz solution, projected along the z-axis')
fig.add_scatter(y=X, name='X')
fig.add_scatter(y=y, name='y')
fig.show()

In [ ]:
period = 10
periodicSampler = Resampler(period=period)

X_sampled, y_sampled = periodicSampler.fit_transform_resample(X, y)

In [ ]:
fig = px.line(title='Trajectory of the Lorenz solution, projected along the z-axis and resampled every 10h')
fig.add_scatter(y=X_sampled.flatten(), name='X_sampled')
fig.add_scatter(y=y_sampled, name='y_sampled')
fig.show()

In [ ]:
max_time_delay = 3
max_embedding_dimension = 10
stride = 1
optimal_time_delay, optimal_embedding_dimension = takens_embedding_optimal_parameters(
    X_sampled, max_time_delay, max_embedding_dimension, stride=stride
    )

print(f"Optimal embedding time delay based on mutual information: {optimal_time_delay}")
print(f"Optimal embedding dimension based on false nearest neighbors: {optimal_embedding_dimension}")

Optimal embedding time delay based on mutual information: 3
Optimal embedding dimension based on false nearest neighbors: 10


In [ ]:
window_size = 41
window_stride = 5
SW = SlidingWindow(size=window_size, stride=window_stride)

X_windows, y_windows = SW.fit_transform_resample(X_sampled, y_sampled)

TE = TakensEmbedding(time_delay=optimal_time_delay, dimension=optimal_embedding_dimension, stride=stride)
X_embedded = TE.fit_transform(X_windows)

In [ ]:
window_number = 3
TE.plot(X_embedded, sample=window_number)

In [ ]:
embedded_begin, embedded_end = SW.slice_windows(X_windows)[window_number]
window_indices = np.arange(embedded_begin, embedded_end + optimal_time_delay * (optimal_embedding_dimension - 1))
fig = px.line(title=f"Resampled Lorenz solution over sliding window {window_number}")
fig.add_scatter(x=window_indices, y=X_sampled[window_indices], name="X_sampled")
fig.show()

In [ ]:
homology_dimensions = (0, 1, 2)
WA = WeakAlphaPersistence(homology_dimensions=homology_dimensions)

X_diagrams = WA.fit_transform(X_embedded)

In [ ]:
WA.plot(X_diagrams, sample=window_number)

In [ ]:
# Steps of the Pipeline
steps = [('sampling', periodicSampler),
         ('window', SW),
         ('embedding', TE),
         ('diagrams', WA)]

# Define the Pipeline
pipeline = Pipeline(steps)

# Run the pipeline
X_diagrams = pipeline.fit_transform(X)

In [ ]:
pipeline[-1].plot(X_diagrams, sample=window_number)

In [ ]:
diagramScaler = Scaler()

X_scaled = diagramScaler.fit_transform(X_diagrams)

diagramScaler.plot(X_scaled, sample=window_number)

In [ ]:
diagramFiltering = Filtering(epsilon=0.1, homology_dimensions=(1, 2))

X_filtered = diagramFiltering.fit_transform(X_scaled)

diagramFiltering.plot(X_filtered, sample=window_number)

In [ ]:
steps_new = [
    ('scaler', diagramScaler),
    ('filtering', diagramFiltering)
]

pipeline_filter = Pipeline(steps + steps_new)

X_filtered = pipeline_filter.fit_transform(X)

In [ ]:
PE = PersistenceEntropy()

X_persistence_entropy = PE.fit_transform(X_scaled)

In [ ]:
fig = px.line(title='Persistence entropies, indexed by sliding window number')
for dim in range(X_persistence_entropy.shape[1]):
    fig.add_scatter(y=X_persistence_entropy[:, dim], name=f"PE in homology dimension {dim}")
fig.show()

In [ ]:
BC = BettiCurve()

X_betti_curves = BC.fit_transform(X_scaled)

BC.plot(X_betti_curves, sample=window_number)

In [ ]:
p_L = 2
n_layers = 5
PD = PairwiseDistance(metric='landscape',
                      metric_params={'p': p_L, 'n_layers': n_layers, 'n_bins': 1000},
                      order=None)

X_distance_L = PD.fit_transform(X_diagrams)
X_distance_L.shape

(91, 91, 3)

In [ ]:
plot_heatmap(X_distance_L[:, :, 0], colorscale='blues')

In [ ]:
p_W = 2
PD = PairwiseDistance(metric='wasserstein',
                      metric_params={'p': p_W, 'delta': 0.1},
                      order=None)

X_distance_W = PD.fit_transform(X_diagrams)

In [ ]:
plot_heatmap(X_distance_W[:, :, 0], colorscale='blues')

In [ ]:
n_neighbors = 2
kNN = KNeighborsGraph(n_neighbors=n_neighbors)

X_kNN = kNN.fit_transform(X_embedded)

In [ ]:
GGD = GraphGeodesicDistance()

GGD.fit_transform_plot(X_kNN, sample=window_number);

In [ ]:
plot_heatmap(pairwise_distances(X_embedded[window_number]), colorscale='blues')

In [ ]:
# Steps of the Pipeline
steps = [
    ('sampling', periodicSampler),
    ('window', SW),
    ('embedding', TE),
    ('kNN_graph', kNN),
    ('graph_geo_distance', GGD),
    ('diagrams', VietorisRipsPersistence(metric='precomputed',
                                         homology_dimensions=homology_dimensions))
    ]

# Define the Pipeline
pipeline = Pipeline(steps)

# Run the pipeline
X_diagrams = pipeline.fit_transform(X)

In [ ]:
pipeline[-1].plot(X_diagrams, sample=window_number)

In [ ]:
X_diagrams.shape

(91, 18, 3)

In [ ]:
period = 10
periodicSampler = Resampler(period=period)

X_sampled, y_sampled = periodicSampler.fit_transform_resample(X, y)

In [ ]:
from gtda.time_series import Resampler, SlidingWindow, takens_embedding_optimal_parameters, \
    TakensEmbedding, PermutationEntropy

In [ ]:
f_df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Datetime,Close_fillna
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,2017-01-04,2742.0
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,2017-01-04,571.0
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,2017-01-04,3210.0
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,2017-01-04,1550.0
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,2017-01-04,3330.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816,2021-12-03,528.0
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478,2021-12-03,794.0
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302,2021-12-03,1645.0
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098,2021-12-03,2389.0


In [ ]:
df_1301 = f_df[f_df['SecuritiesCode'] == 1301]

In [ ]:
df_9990 = f_df[f_df['SecuritiesCode'] == 9990]

In [ ]:
# X_jpx = f_df['Close_fillna'].values
# y_jpx = f_df['Target'].fillna(0).values

In [225]:
X_jpx = df_9990['Close_fillna'].values
y_jpx = df_9990['Target'].fillna(0).values

In [226]:
X_sampled = X_jpx
y_sampled = y_jpx

In [227]:
# period = 10
# periodicSampler = Resampler(period=period)

# X_sampled, y_sampled = periodicSampler.fit_transform_resample(X_jpx, y_jpx)

In [228]:
X_sampled.shape

(1202,)

In [229]:
max_time_delay = 3
max_embedding_dimension = 10
stride = 1
optimal_time_delay, optimal_embedding_dimension = takens_embedding_optimal_parameters(
    X_sampled, max_time_delay, max_embedding_dimension, stride=stride
    )


In [230]:
window_size = 41
window_stride = 5
SW = SlidingWindow(size=window_size, stride=window_stride)

X_windows, y_windows = SW.fit_transform_resample(X_sampled, y_sampled)

TE = TakensEmbedding(time_delay=optimal_time_delay, dimension=optimal_embedding_dimension, stride=stride)
X_embedded = TE.fit_transform(X_windows)

In [231]:
X_windows.shape

(233, 41)

In [232]:
window_number = 3
TE.plot(X_embedded, sample=window_number)

In [233]:
embedded_begin, embedded_end = SW.slice_windows(X_windows)[window_number]
window_indices = np.arange(embedded_begin, embedded_end + optimal_time_delay * (optimal_embedding_dimension - 1))
fig = px.line(title=f"Resampled Lorenz solution over sliding window {window_number}")
fig.add_scatter(x=window_indices, y=X_sampled[window_indices], name="X_sampled")
fig.show()

In [234]:
homology_dimensions = (0, 1, 2)
WA = WeakAlphaPersistence(homology_dimensions=homology_dimensions)

X_diagrams = WA.fit_transform(X_embedded)

In [235]:
WA.plot(X_diagrams, sample=window_number)

In [236]:
# Steps of the Pipeline
steps = [('sampling', periodicSampler),
         ('window', SW),
         ('embedding', TE),
         ('diagrams', WA)]

# Define the Pipeline
pipeline = Pipeline(steps)

# Run the pipeline
X_diagrams = pipeline.fit_transform(X)

In [237]:
pipeline[-1].plot(X_diagrams, sample=window_number)

In [238]:
diagramScaler = Scaler()

X_scaled = diagramScaler.fit_transform(X_diagrams)

diagramScaler.plot(X_scaled, sample=window_number)

In [239]:
diagramFiltering = Filtering(epsilon=0.1, homology_dimensions=(1, 2))

X_filtered = diagramFiltering.fit_transform(X_scaled)

diagramFiltering.plot(X_filtered, sample=window_number)

In [240]:
steps_new = [
    ('scaler', diagramScaler),
    ('filtering', diagramFiltering)
]

pipeline_filter = Pipeline(steps + steps_new)

X_filtered = pipeline_filter.fit_transform(X)

In [241]:
PE = PersistenceEntropy()

X_persistence_entropy = PE.fit_transform(X_scaled)

In [242]:
fig = px.line(title='Persistence entropies, indexed by sliding window number')
for dim in range(X_persistence_entropy.shape[1]):
    fig.add_scatter(y=X_persistence_entropy[:, dim], name=f"PE in homology dimension {dim}")
fig.show()

In [243]:
BC = BettiCurve()

X_betti_curves = BC.fit_transform(X_scaled)

BC.plot(X_betti_curves, sample=window_number)

In [244]:
p_L = 2
n_layers = 5
PD = PairwiseDistance(metric='landscape',
                      metric_params={'p': p_L, 'n_layers': n_layers, 'n_bins': 1000},
                      order=None)

X_distance_L = PD.fit_transform(X_diagrams)
X_distance_L.shape

(91, 91, 3)

In [245]:
plot_heatmap(X_distance_L[:, :, 0], colorscale='blues')

In [203]:
X_distance_L.shape

(91, 91, 3)